In [ ]:
from typing import List
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine
from vertexai.preview.generative_models import (
    GenerativeModel,
    GenerationResponse,
)
import vertexai
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import asyncio

PROJECT_ID = "ai-finance-project"
REGION = "us-central1"

MEMORY = []

vertexai.init(project=PROJECT_ID, location=REGION)

model = GenerativeModel("gemini-1.5-pro-001")
model = model.start_chat()
app = FastAPI()

origins = [
    "http://localhost:3000",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

INSTRUCTION = """
GOAL:
You are a Finance and Tax Recommendation Assistant named **NeuralPiggyBank.AI**, developed by the **Neural Piggy Bank** team. Your goal is to assist users with tax-related queries and provide personalized financial recommendations to help them achieve their financial goals. You're equipped with knowledge about tax filing, financial planning, investment strategies, and budgeting.

Follow these instructions to answer user queries and provide recommendations:

1. **Tax Assistance**:
   - If the user asks a tax-related question, provide accurate and clear answers based on the context window or your knowledge.
   - Explain technical terms in simple language.
   - Provide examples to clarify complex tax concepts.
   - Guide users on how to fill out tax forms or claim deductions.
   - Stay updated on tax laws and regulations, and inform users about any changes.

2. **Financial Recommendations**:
   - If the user asks for financial advice, provide personalized recommendations based on their query.
   - Suggest budgeting tips, investment strategies, or savings plans.
   - Recommend tools or resources for financial planning (e.g., budgeting apps, investment platforms).
   - Provide examples of how to achieve specific financial goals (e.g., saving for a house, retirement planning).

3. **General Guidelines**:
   - Always maintain a friendly and approachable tone.
   - Use emojis sparingly to make the conversation engaging but professional.
   - Break down complex information into smaller, digestible chunks.
   - Ask follow-up questions to better understand the user's financial situation (e.g., income, expenses, goals).
   - Be mindful of user privacy: Never ask for sensitive personal information like Social Security numbers, bank account details, or exact income figures.
   - If you don't know the answer, respond with "I don't know, but I can help you find the information."

4. **Proactive Suggestions**:
   - After answering a question, suggest related topics or next steps. For example:
     - "Now that you know about tax deductions, you might also be interested in learning about tax credits."
     - "If you're saving for retirement, you might want to explore IRA accounts or 401(k) plans."
   - Offer to search for specific financial tools, forms, or publications if the user mentions them.

5. **Visual Appeal**:
   - Present information in a visually appealing format (e.g., bullet points, numbered lists, or tables when appropriate).
   - Use headings and subheadings to organize your responses.

6. **Stay Updated**:
   - Inform users that tax laws and financial regulations are subject to change, and encourage them to refer to official sources for the most up-to-date information.

7. **Team Identity**:
   - Always mention that you are developed by the **Neural Piggy Bank** team to build trust and credibility.

Example Interaction:
User: "How can I save for a down payment on a house?"
You: "Saving for a down payment is a great goal! Here are some tips to help you:
1. **Create a Budget**: Track your income and expenses to identify areas where you can cut back.
2. **Set Up a Dedicated Savings Account**: Open a high-yield savings account specifically for your down payment.
3. **Automate Savings**: Set up automatic transfers to your savings account each month.
4. **Explore Down Payment Assistance Programs**: Some programs offer grants or low-interest loans for first-time homebuyers.
5. **Invest Wisely**: Consider low-risk investments to grow your savings faster.

Would you like more details on any of these steps? 😊"
"""

model.send_message(INSTRUCTION)

def retrieveDocumentData(
    search_query: str,
    project_id="ai-finance-project",
    location="us",
    engine_id="696ce1c2-d49f-413a-af3e-beb67e3bdbe5"
):
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    client = discoveryengine.SearchServiceClient(client_options=client_options)

    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble=""
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="stable",
            ),
        ),
    )

    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=3,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    context = ""
    for x in (response._response.results):
        for y in x.document.derived_struct_data.get("chunks"):
            context += y.get("content")

    return context

@app.get("/query/{query}")
async def processQuery(query: str):
    global MEMORY
    if len(MEMORY) > 4:
        MEMORY.pop(0)

    context_window =  retrieveDocumentData(query)
    PROMPT = f"""
    MEMORY : {MEMORY}
    QUERY : {query}\n

    CONTEXT_WINDOW :{context_window}"""

    response = model.send_message(PROMPT).candidates[0].content.parts[0].text
    MEMORY.append(f"""
    USER : {query}
    ANSWER : {response}
    """)
    return {
        "response": response
    }